# NOTE: Requires specific version of firmware

In [1]:
# Import libraries
%load_ext autoreload
%autoreload 2

%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Board libraries
from qick import *
import Pyro4

# Pyro communicates with board
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Thu Aug 29 17:24:39 2024

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [9]:
prog = QickProgram(soccfg)

prog.regwi(0, 2, 253) # Write 253 to p0,r2

prog.trigger(pins=[0,1], width=15) # Produce digital pulse

prog.wait_all()

prog.read(2, 0, "lower", 1) # Read digital input and write to register p0,r1
prog.memwi(0, 1, 123) # Write value to memory

prog.condj(0, 1, '<', 2, "JUMP_LOC") # Compare digital input value to 253
prog.trigger(pins=[0,1]) # Conditionally trigger a second pulse
prog.label("JUMP_LOC")

prog.end()

prog.config_all(soc) # Load the assembly onto the board
soc.tproc.start() # Start tproc program execution

result = [np.int32(soc.tproc.single_read(addr=123))]
print("Values read from tproc input:", result)

Values read from tproc input: [254]


In [161]:
print(prog)


// Program

          regwi 0, $2, 253;
          regwi 0, $16, 3;                      //out = 0b0000000000000011
          seti 7, 0, $16, 0;                    //ch =0 out = $16 @t = 0
          seti 7, 0, $0, 15;                    //ch =0 out = 0 @t = 0
          waiti 0, 0;
          read 2, 0, lower $1;
          memwi 0, $1, 123;
          condj 0, $1, <, $2, @JUMP_LOC;
          regwi 0, $16, 3;                      //out = 0b0000000000000011
          seti 7, 0, $16, 0;                    //ch =0 out = $16 @t = 0
          seti 7, 0, $0, 10;                    //ch =0 out = 0 @t = 0
JUMP_LOC: end ;
